### Set Up Packages and Global Settings

In [2]:
import pytz
import random
import time

from datetime import datetime

import fastf1 as ff1
import pandas as pd

pd.set_option("display.max_columns", None)

### Explore Schedule

In [ ]:
schedule = ff1.get_event_schedule(1994)
schedule.RoundNumber.to_list()

### Explore Session Object

In [ ]:
session_quali = ff1.get_session(2020, 1, "Q")

session_quali.load()
session_quali.event

In [ ]:
session_quali = ff1.get_session(2020, 1, 'R')

session_quali.load()
session_quali.results

### Explore Laps

In [ ]:
lap_data = session_number.laps
print(lap_data)

In [ ]:
lap_data.pick_team('McLaren').pick_fastest()

In [ ]:
lap_data.pick_drivers(['PIA', 'NOR'])

### Compare DriverId field to names

In [ ]:
df_all = pd.DataFrame()

list_years = list(range(1994, 1995 + 1))

max_retries = 5

for year in list_years:
    print(f"Pulling data for {year}")

    schedule = ff1.get_event_schedule(year)
    rounds_all = schedule.RoundNumber.to_list()
    rounds_races = [round for round in rounds_all if round > 0]

    for round in rounds_races:
        for attempt in range(max_retries):
            try:
                now_utc = datetime.now(pytz.timezone("UTC"))
                now_cst = now_utc.astimezone(pytz.timezone("America/Chicago"))

                print(f"Pulling data for round {round} at {now_cst}...")

                session_quali = ff1.get_session(year, round, "Q")
                session_quali.load()
                df_quali = session_quali.results[["DriverId", "LastName", "FirstName"]]

                df_all = df_all.append(df_quali, ignore_index=True)
                time.sleep(5)

                session_race = ff1.get_session(year, round, "R")
                session_race.load()
                df_race = session_race.results[["DriverId", "LastName", "FirstName"]]

                df_all = df_all.append(df_race, ignore_index=True)
                time.sleep(5)

                df_all = df_all.drop_duplicates().reset_index(drop=True)

                # If no exception was raised, break the loop
                break
            except Exception as e:
                wait_time = (2**attempt) + random.random()
                print(
                    f"Could not load data for round {round} due to {e}. Retrying in {wait_time} seconds."
                )
                time.sleep(wait_time)
                continue

df_unique = df_all.sort_values(by="DriverId")

### Develop Approach for Quali Results Object

In [11]:
session_quali = ff1.get_session(1994, 1, "Q")

session_quali.load()
df_raw = session_quali.results[
    ["Q1", "Q2", "Q3", "DriverId", "LastName", "FirstName", "TeamName", "Position"]
]

df_processed = df_raw.melt(
    id_vars=["DriverId", "LastName", "FirstName", "TeamName", "Position"],
    value_vars=["Q1", "Q2", "Q3"],
    var_name="session",
    value_name="time",
)

df_processed.columns = df_processed.columns.str.lower()

df_processed["originalposition"] = df_processed["position"]
df_processed["position"] = df_processed.groupby("session")["time"].rank(
    method="min", ascending=True
)
df_processed["position"] = df_processed["position"].fillna(
    df_processed["originalposition"]
)

df_processed = df_processed.drop(columns=["originalposition"])

df_processed.rename(
    columns={
        "driverid": "id_driver",
        "lastname": "name_driver_last",
        "firstname": "name_driver_first",
        "teamname": "name_team",
    },
    inplace=True,
)
df_final = df_processed[
    [
        "id_driver",
        "name_driver_last",
        "name_driver_first",
        "name_team",
        "session",
        "position",
        "time",
    ]
]

df_final

core           INFO 	Loading data for Brazilian Grand Prix - Qualifying [v3.1.6]
req            INFO 	No cached data found for session_info. Loading data...


_api           INFO 	Fetching session info data...
logger      WARNING 	Failed to load session info data!
core        WARNING 	Cannot load laps, telemetry, weather, and message data because the relevant API is not supported for this session.
core           INFO 	Finished loading data for 26 drivers: ['2', '5', '27', '0', '30', '10', '29', '7', '6', '3', '9', '4', '20', '14', '23', '15', '28', '8', '26', '25', '12', '24', '19', '11', '34', '31']


,id_driver,name_driver_last,name_driver_first,name_team,session,position,time
0,senna,Senna,Ayrton,Williams,Q1,1.0,0 days 00:01:15.962000
1,michael_schumacher,Schumacher,Michael,Benetton,Q1,2.0,0 days 00:01:16.290000
2,alesi,Alesi,Jean,Ferrari,Q1,3.0,0 days 00:01:17.385000
3,damon_hill,Hill,Damon,Williams,Q1,4.0,0 days 00:01:17.554000
4,frentzen,Frentzen,Heinz-Harald,Sauber,Q1,5.0,0 days 00:01:17.806000
...,...,...,...,...,...,...,...
73,alboreto,Alboreto,Michele,Minardi,Q3,22.0,NaT
74,beretta,Beretta,Olivier,Larrousse,Q3,23.0,NaT
75,lamy,Lamy,Pedro,Team Lotus,Q3,24.0,NaT
76,gachot,Gachot,Bertrand,Pacific,Q3,25.0,NaT


### Develop Approach for Race Results Object

In [12]:
session_race = ff1.get_session(1950, 1, "R")

session_race.load()
df_raw = session_race.results[
    [
        "DriverId",
        "LastName",
        "FirstName",
        "TeamName",
        "ClassifiedPosition",
        "Time",
    ]
]

df_processed = df_raw.copy().reset_index(drop=True)

df_processed["session"] = "Race"

df_processed.columns = df_processed.columns.str.lower()
df_processed.rename(
    columns={
        "classifiedposition": "position",
        "driverid": "id_driver",
        "lastname": "name_driver_last",
        "firstname": "name_driver_first",
        "teamname": "name_team",
    },
    inplace=True,
)
df_processed.loc[1:, "time"] = (
    df_processed.loc[1:, "time"] + df_processed.loc[0, "time"]
)
df_final = df_processed[
    [
        "id_driver",
        "name_driver_last",
        "name_driver_first",
        "name_team",
        "session",
        "position",
        "time",
    ]
]

df_final

core           INFO 	Loading data for British Grand Prix - Race [v3.1.6]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
logger      WARNING 	Failed to load session info data!
core        WARNING 	Session results contain duplicate entries for driver(s) ['10', '9']
core        WARNING 	Cannot load laps, telemetry, weather, and message data because the relevant API is not supported for this session.
core           INFO 	Finished loading data for 21 drivers: ['2', '3', '4', '14', '15', '12', '11', '16', '6', '10', '18', '1', '23', '21', '5', '24', '20', '19', '17', '9', '8']


,id_driver,name_driver_last,name_driver_first,name_team,session,position,time
0,farina,Farina,Nino,Alfa Romeo,Race,1,0 days 02:13:23.600000
1,fagioli,Fagioli,Luigi,Alfa Romeo,Race,2,0 days 02:13:26.200000
2,reg_parnell,Parnell,Reg,Alfa Romeo,Race,3,0 days 02:14:15.600000
3,cabantous,Cabantous,Yves,Talbot-Lago,Race,4,NaT
4,rosier,Rosier,Louis,Talbot-Lago,Race,5,NaT
5,gerard,Gerard,Bob,ERA,Race,6,NaT
6,harrison,Harrison,Cuth,ERA,Race,7,NaT
7,etancelin,Étancelin,Philippe,Talbot-Lago,Race,8,NaT
8,hampshire,Hampshire,David,Maserati,Race,9,NaT
9,shawe_taylor,Shawe Taylor,Brian,Maserati,Race,10,NaT


In [ ]:
df_raw = pd.DataFrame(
    session_quali.event[["RoundNumber", "Location", "EventDate", "Country"]]
)

df_processed = df_raw.T.reset_index(drop=True)
df_processed.columns = df_processed.columns.str.lower()
df_processed.rename(
    columns={
        "roundnumber": "round",
        "location": "circuit_name",
        "country": "circuit_country",
    },
    inplace=True,
)
df_processed["year"] = df_processed["eventdate"].dt.year

df_final = df_processed[
    [
        "year",
        "round",
        "circuit_name",
        "circuit_country"
    ]
]

df_final